# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:885
  ^(!Matched::Regex, ::Integer) at regex.jl:712
  ^(!Matched::Missing, ::Integer) at missing.jl:155
  ...

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [4]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [5]:
foo("hello", "hi!")

My inputs x and y are both strings!


In [6]:
foo(3, 4)

MethodError: MethodError: no method matching foo(::Int64, ::Int64)

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [7]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [8]:
foo(3, 4)

My inputs x and y are both integers!


Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [9]:
foo("hello", "hi!")

My inputs x and y are both strings!


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(y::String, y::String)
```
Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [10]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[7]:1
[2] foo(x::String, y::String) in Main at In[4]:1

Aside: how many methods do you think there are for addition?

In [11]:
methods(+)

# 166 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:286
[2] +(x::Bool, y::Bool) in Base at bool.jl:96
[3] +(x::Bool) in Base at bool.jl:93
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:104
[5] +(x::Bool, z::Complex) in Base at complex.jl:293
[6] +(a::Float16, b::Float16) in Base at float.jl:398
[7] +(x::Float32, y::Float32) in Base at float.jl:400
[8] +(x::Float64, y::Float64) in Base at float.jl:401
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:287
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:301
[11] +(::Missing, ::Missing) in Base at missing.jl:114
[12] +(::Missing) in Base at missing.jl:100
[13] +(::Missing, ::Number) in Base at missing.jl:115
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:107
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:414
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:515
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:514
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:513
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:473
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:521
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:527
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:564
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:557
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:551
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:410
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:379
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:386
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:394
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:402
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/periods.jl:341
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/periods.jl:339
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/periods.jl:369
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /home/abolfazl/prog/install-dir/julia-1.4.1/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:76
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /home

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [12]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[7]:1

Let's see what happens when we use `@which` with the addition operator!

In [15]:
@which 3.0 + 3.0

+(x::Float64, y::Float64) in Base at float.jl:401

And we can continue to add other methods to our generic function `foo`. Let's add one that takes the ***abstract type*** `Number`, which includes subtypes such as `Int`, `Float64`, and other objects you would think of as numbers:

In [16]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")

foo (generic function with 3 methods)

This method for `foo` will work on, for example, floating point numbers:

In [17]:
foo(3.0, 4.0)

My inputs x and y are both numbers!


We can also add a fallback, duck-typed method for `foo` that takes inputs of any type:

In [18]:
foo(x, y) = println("I accept inputs of any type!")

foo (generic function with 4 methods)

Given the methods we've already written for `foo` so far, this method will be called whenever we pass non-numbers to `foo`:

In [19]:
v = rand(3)
foo(v, v)

I accept inputs of any type!


### Exercises

#### 9.1

Extend the function `foo`, adding a method that takes only one input argument, which is of type `Bool`, and returns "foo with one boolean!"

In [29]:
foo(x::Bool) = "foo with one boolean!"

foo (generic function with 5 methods)

#### 9.2

Check that the method being dispatched when you execute 
```julia
foo(true)
```
is the one you wrote.

In [30]:
foo(true)

"foo with one boolean!"

In [31]:
@assert foo(true) == "foo with one boolean!"

Please click on `Validate` on the top, once you are done with the exercises.

In [32]:
# "hello" + "world!"

MethodError: MethodError: no method matching +(::String, ::String)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529

In [34]:
add_strings(x::String, y::String) = string(x, y)

add_strings (generic function with 1 method)

In [37]:
add_strings("hi", "jey")

"hijey"